# 06 — Reentrenamiento continuo (Auto + Human-in-the-loop)

**Objetivo:** incorporar nuevas imágenes al dataset y reentrenar el modelo de forma controlada.

Se manejan dos fuentes de nueva data:

- **AUTO** (`data/new/auto/`): generada por la app automáticamente (pseudo-labels)
- **HUMAN** (`data/new/human/`): corregida o creada manualmente (labels confiables)

Reglas:
1) Se ingiere primero **HUMAN** (prioridad)
2) Se ingiere después **AUTO** pero con filtro mínimo (para no meter ruido)
3) Se realiza fine-tuning desde `models/yolo_best.pt`
4) Se actualiza el modelo final y se registra en MLflow


In [17]:
# --- IMPORTS ---
from pathlib import Path
import os
import json
import shutil
import time
from datetime import datetime
from typing import Dict, List, Tuple
import numpy as np


In [18]:
# --- RUTAS PRINCIPALES ---
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

YOLO_DATASET_DIR = PROJECT_ROOT / "data" / "processed" / "yolo_dataset"
DATA_YAML_PATH = YOLO_DATASET_DIR / "data.yaml"

TRAIN_IMAGES_DIR = YOLO_DATASET_DIR / "images" / "train"
TRAIN_LABELS_DIR = YOLO_DATASET_DIR / "labels" / "train"

MODELS_DIR = PROJECT_ROOT / "models"
CURRENT_MODEL_PATH = MODELS_DIR / "yolo_best.pt"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"
CLASS_MAP_PATH = ARTIFACTS_DIR / "class_map.json"

RUNS_DIR = PROJECT_ROOT / "runs"
MLRUNS_DIR = PROJECT_ROOT / "mlruns"

INGEST_LOG_DIR = ARTIFACTS_DIR / "retraining_ingest_logs"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("CURRENT_MODEL_PATH:", CURRENT_MODEL_PATH)
print("YOLO_DATASET_DIR:", YOLO_DATASET_DIR)
print("RUNS_DIR:", RUNS_DIR)
print("MLRUNS_DIR:", MLRUNS_DIR)


PROJECT_ROOT: c:\Users\Johnny\Desktop\IA
CURRENT_MODEL_PATH: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt
YOLO_DATASET_DIR: c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset
RUNS_DIR: c:\Users\Johnny\Desktop\IA\runs
MLRUNS_DIR: c:\Users\Johnny\Desktop\IA\mlruns


In [19]:
# --- RUTAS NUEVA DATA (AUTO vs HUMAN) ---
NEW_DATA_DIR = PROJECT_ROOT / "data" / "new"

AUTO_IMAGES_DIR = NEW_DATA_DIR / "auto" / "images"
AUTO_LABELS_DIR = NEW_DATA_DIR / "auto" / "labels"

HUMAN_IMAGES_DIR = NEW_DATA_DIR / "human" / "images"
HUMAN_LABELS_DIR = NEW_DATA_DIR / "human" / "labels"

for d in [
    TRAIN_IMAGES_DIR, TRAIN_LABELS_DIR,
    RUNS_DIR, MLRUNS_DIR, INGEST_LOG_DIR,
    AUTO_IMAGES_DIR, AUTO_LABELS_DIR,
    HUMAN_IMAGES_DIR, HUMAN_LABELS_DIR
]:
    d.mkdir(parents=True, exist_ok=True)

print("AUTO:", AUTO_IMAGES_DIR)
print("HUMAN:", HUMAN_IMAGES_DIR)


AUTO: c:\Users\Johnny\Desktop\IA\data\new\auto\images
HUMAN: c:\Users\Johnny\Desktop\IA\data\new\human\images


In [20]:
# --- VALIDACIONES ---
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(YOLO_DATASET_DIR, "dataset YOLO (Notebook 02)")
assert_exists(DATA_YAML_PATH, "data.yaml")
assert_exists(CURRENT_MODEL_PATH, "modelo actual yolo_best.pt (Notebook 03)")
assert_exists(CONFIG_SNAPSHOT_PATH, "config_snapshot.json")
assert_exists(CLASS_MAP_PATH, "class_map.json")

print("Validaciones OK.")


Validaciones OK.


In [21]:
# --- CARGA DE CONFIGURACIÓN ---
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

with open(CLASS_MAP_PATH, "r", encoding="utf-8") as f:
    class_map = json.load(f)

target_classes = cfg["target_classes"]
imgsz = int(cfg["img_size"])
batch = int(cfg["batch_size"])
seed = int(cfg["seed"])

print("Clases:", target_classes)
print("imgsz:", imgsz, "batch:", batch, "seed:", seed)


Clases: ['car', 'airplane', 'truck']
imgsz: 540 batch: 20 seed: 42


## Validación de labels YOLO

Formato esperado por línea:
`<class_id> <x_center> <y_center> <w> <h>` (normalizados en [0,1])

- **HUMAN:** se acepta si el formato es válido
- **AUTO:** se acepta si el formato es válido y pasa un filtro mínimo (ej. al menos 1 bbox)


In [22]:
# --- UTILIDADES: ESCANEO Y VALIDACIÓN ---
IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp"}

def list_pairs(images_dir: Path, labels_dir: Path) -> List[Tuple[Path, Path]]:
    pairs = []
    for img_path in images_dir.iterdir():
        if not img_path.is_file():
            continue
        if img_path.suffix.lower() not in IMG_EXTS:
            continue
        lbl_path = labels_dir / f"{img_path.stem}.txt"
        if lbl_path.exists():
            pairs.append((img_path, lbl_path))
    return pairs

def validate_yolo_label_file(label_path: Path, num_classes: int) -> Tuple[bool, str]:
    try:
        lines = [ln.strip() for ln in label_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
        if len(lines) == 0:
            return False, "label vacío"
        for ln in lines:
            parts = ln.split()
            if len(parts) != 5:
                return False, "formato incorrecto (no hay 5 columnas)"
            cls = int(float(parts[0]))
            vals = [float(x) for x in parts[1:]]
            if cls < 0 or cls >= num_classes:
                return False, "class_id fuera de rango"
            if any((v < 0.0 or v > 1.0) for v in vals):
                return False, "valores bbox fuera de [0,1]"
            if vals[2] <= 0 or vals[3] <= 0:
                return False, "w/h no pueden ser 0 o negativas"
        return True, "ok"
    except Exception as e:
        return False, f"error leyendo label: {e}"

AUTO_MIN_BOXES = 1

def auto_is_acceptable(label_path: Path) -> bool:
    lines = [ln.strip() for ln in label_path.read_text(encoding="utf-8").splitlines() if ln.strip()]
    return len(lines) >= AUTO_MIN_BOXES


In [23]:
# --- ESCANEAR NUEVA DATA (HUMAN Y AUTO) ---
human_pairs = list_pairs(HUMAN_IMAGES_DIR, HUMAN_LABELS_DIR)
auto_pairs = list_pairs(AUTO_IMAGES_DIR, AUTO_LABELS_DIR)

print("HUMAN pairs:", len(human_pairs))
print("AUTO pairs:", len(auto_pairs))


HUMAN pairs: 0
AUTO pairs: 0


In [24]:
# --- VALIDAR NUEVA DATA ---
valid_human, invalid_human = [], []
for img, lbl in human_pairs:
    ok, msg = validate_yolo_label_file(lbl, num_classes=len(target_classes))
    if ok:
        valid_human.append((img, lbl))
    else:
        invalid_human.append((img, lbl, msg))

valid_auto, invalid_auto = [], []
for img, lbl in auto_pairs:
    ok, msg = validate_yolo_label_file(lbl, num_classes=len(target_classes))
    if ok and auto_is_acceptable(lbl):
        valid_auto.append((img, lbl))
    else:
        reason = msg if not ok else "auto_filter_rejected"
        invalid_auto.append((img, lbl, reason))

print("Valid HUMAN:", len(valid_human), "| Invalid HUMAN:", len(invalid_human))
print("Valid AUTO:", len(valid_auto), "| Invalid AUTO:", len(invalid_auto))


Valid HUMAN: 0 | Invalid HUMAN: 0
Valid AUTO: 0 | Invalid AUTO: 0


In [25]:
# --- LOG DE INGESTA ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
INGEST_LOG_PATH = INGEST_LOG_DIR / f"ingest_{timestamp}.json"

log = {
    "timestamp": timestamp,
    "human_detected": len(human_pairs),
    "auto_detected": len(auto_pairs),
    "human_valid": len(valid_human),
    "auto_valid": len(valid_auto),
    "human_invalid": [
        {"image": str(a), "label": str(b), "reason": r} for (a, b, r) in invalid_human
    ],
    "auto_invalid": [
        {"image": str(a), "label": str(b), "reason": r} for (a, b, r) in invalid_auto
    ],
    "auto_min_boxes": AUTO_MIN_BOXES
}

INGEST_LOG_PATH.write_text(json.dumps(log, indent=2), encoding="utf-8")
print("Log guardado en:", INGEST_LOG_PATH)


Log guardado en: c:\Users\Johnny\Desktop\IA\artifacts\retraining_ingest_logs\ingest_20260129_131548.json


## Ingesta al train

Se copian los pares válidos al train con prefijos únicos para evitar colisiones:

- `human_<timestamp>_...`
- `auto_<timestamp>_...`

No se borra nada de `data/new/` automáticamente.


In [26]:
# --- COPIA SEGURA (CON PREFIJO ÚNICO) ---
def safe_copy_pair(img_path: Path, lbl_path: Path, prefix: str) -> Tuple[Path, Path]:
    new_img_name = f"{prefix}_{img_path.name}"
    new_lbl_name = f"{prefix}_{lbl_path.name}"

    dst_img = TRAIN_IMAGES_DIR / new_img_name
    dst_lbl = TRAIN_LABELS_DIR / new_lbl_name

    shutil.copy2(img_path, dst_img)
    shutil.copy2(lbl_path, dst_lbl)

    return dst_img, dst_lbl

ingested = []

human_prefix = f"human_{timestamp}"
auto_prefix = f"auto_{timestamp}"

for img, lbl in valid_human:
    ingested.append(safe_copy_pair(img, lbl, human_prefix))

for img, lbl in valid_auto:
    ingested.append(safe_copy_pair(img, lbl, auto_prefix))

print("Total incorporado a train:", len(ingested))
print("HUMAN incorporado:", len(valid_human))
print("AUTO incorporado:", len(valid_auto))


Total incorporado a train: 0
HUMAN incorporado: 0
AUTO incorporado: 0


In [27]:
# --- FIX MLFLOW URI (WINDOWS) ANTES DEL TRAIN ---
MLRUNS_DIR = MLRUNS_DIR.resolve()
tracking_uri = MLRUNS_DIR.as_uri()

os.environ["MLFLOW_TRACKING_URI"] = tracking_uri
os.environ["MLFLOW_REGISTRY_URI"] = tracking_uri

print("MLFLOW_TRACKING_URI =", os.environ["MLFLOW_TRACKING_URI"])
print("MLFLOW_REGISTRY_URI =", os.environ["MLFLOW_REGISTRY_URI"])


MLFLOW_TRACKING_URI = file:///C:/Users/Johnny/Desktop/IA/mlruns
MLFLOW_REGISTRY_URI = file:///C:/Users/Johnny/Desktop/IA/mlruns


In [28]:
# --- ENTRENAMIENTO CONTINUO (FINE-TUNING) ---
from ultralytics import YOLO

RETRAIN_EPOCHS = 10
RETRAIN_BATCH = batch
RETRAIN_IMGSZ = imgsz
RETRAIN_PATIENCE = 3

print("RETRAIN_EPOCHS:", RETRAIN_EPOCHS)
print("RETRAIN_BATCH:", RETRAIN_BATCH)
print("RETRAIN_IMGSZ:", RETRAIN_IMGSZ)
print("RETRAIN_PATIENCE:", RETRAIN_PATIENCE)


RETRAIN_EPOCHS: 10
RETRAIN_BATCH: 20
RETRAIN_IMGSZ: 540
RETRAIN_PATIENCE: 3


In [29]:
# --- EJECUTAR REENTRENAMIENTO SOLO SI HAY DATA NUEVA VÁLIDA ---
if len(ingested) == 0:
    print("No hay nueva data válida para reentrenar (HUMAN/AUTO). Se termina el notebook.")
    retrain_run_name = None
else:
    model = YOLO(str(CURRENT_MODEL_PATH))
    retrain_run_name = f"retrain_{timestamp}"

    retrain_results = model.train(
        data=str(DATA_YAML_PATH),
        imgsz=RETRAIN_IMGSZ,
        batch=RETRAIN_BATCH,
        epochs=RETRAIN_EPOCHS,
        seed=seed,
        project=str(RUNS_DIR),
        name=retrain_run_name,
        pretrained=True,
        patience=RETRAIN_PATIENCE,
        verbose=True
    )

    print("Entrenamiento continuo finalizado:", retrain_run_name)


No hay nueva data válida para reentrenar (HUMAN/AUTO). Se termina el notebook.


In [30]:
# --- UBICACIÓN REAL DEL NUEVO BEST (DINÁMICA) ---
if retrain_run_name is not None:
    save_dir = Path(model.trainer.save_dir)
    NEW_BEST_PATH = save_dir / "weights" / "best.pt"

    if not NEW_BEST_PATH.exists():
        raise FileNotFoundError(f"No se encontró best.pt en: {save_dir}")

    print("Nuevo best:", NEW_BEST_PATH)


In [31]:
# --- ACTUALIZAR MODELO FINAL EN /models (CON BACKUP) ---
if retrain_run_name is not None:
    BACKUP_PATH = MODELS_DIR / f"yolo_best_backup_{timestamp}.pt"
    shutil.copy2(CURRENT_MODEL_PATH, BACKUP_PATH)
    shutil.copy2(NEW_BEST_PATH, CURRENT_MODEL_PATH)

    print("Backup guardado:", BACKUP_PATH)
    print("Modelo actualizado:", CURRENT_MODEL_PATH)


In [32]:
# --- EVALUACIÓN DEL MODELO ACTUALIZADO (VAL) ---
mlflow_metrics = None

if retrain_run_name is not None:
    metrics_obj = model.val(
        data=str(DATA_YAML_PATH),
        imgsz=RETRAIN_IMGSZ,
        batch=RETRAIN_BATCH,
        verbose=False
    )

    d = metrics_obj.results_dict
    mlflow_metrics = {
        "precision": float(d.get("metrics/precision(B)", 0)),
        "recall": float(d.get("metrics/recall(B)", 0)),
        "mAP50": float(d.get("metrics/mAP50(B)", 0)),
        "mAP50_95": float(d.get("metrics/mAP50-95(B)", 0)),
    }

    print("Métricas:", mlflow_metrics)


In [33]:
# --- REGISTRO EN MLFLOW ---
import mlflow

if retrain_run_name is not None:
    mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
    mlflow.set_experiment("coco2017_car_airplane_truck")

    mlflow_params = {
        "stage": "continuous_retraining",
        "retrain_epochs": RETRAIN_EPOCHS,
        "retrain_batch": RETRAIN_BATCH,
        "retrain_imgsz": RETRAIN_IMGSZ,
        "retrain_patience": RETRAIN_PATIENCE,
        "ingested_total": len(ingested),
        "ingested_human": len(valid_human),
        "ingested_auto": len(valid_auto),
        "auto_min_boxes": AUTO_MIN_BOXES,
        "dataset_mode": "limited",
        "classes": ",".join(target_classes),
    }

    with mlflow.start_run(run_name=f"continuous_retrain_{timestamp}"):
        mlflow.log_params(mlflow_params)
        mlflow.log_metrics(mlflow_metrics)

        mlflow.log_artifact(str(CURRENT_MODEL_PATH), artifact_path="model")
        mlflow.log_artifact(str(INGEST_LOG_PATH), artifact_path="ingest_log")
        mlflow.log_artifact(str(CONFIG_SNAPSHOT_PATH), artifact_path="config")
        mlflow.log_artifact(str(CLASS_MAP_PATH), artifact_path="config")
        mlflow.log_artifact(str(DATA_YAML_PATH), artifact_path="dataset")

    print("Run registrado en MLflow.")


## Resultado final

- La nueva data válida se copió al train:
  - `data/processed/yolo_dataset/images/train/`
  - `data/processed/yolo_dataset/labels/train/`

- El reentrenamiento se ejecutó solo si hubo data nueva válida.

- El modelo final se actualizó en:
  - `models/yolo_best.pt`

- Se registró en MLflow con métricas y logs de ingesta.

Sugerencia de workflow:
1) La app guarda auto-labels en `data/new/auto/`
2) Si un caso está mal, lo corriges y lo pones en `data/new/human/`
3) Reentrenas: HUMAN primero, luego AUTO filtrado


In [34]:
# --- RESUMEN FINAL ---
print("Proceso terminado.")
print("Ingestados total:", len(ingested))
print("Ingestados HUMAN:", len(valid_human))
print("Ingestados AUTO:", len(valid_auto))
if mlflow_metrics is not None:
    print("Métricas registradas:", mlflow_metrics)
print("Modelo final:", CURRENT_MODEL_PATH)
print("Log de ingesta:", INGEST_LOG_PATH)


Proceso terminado.
Ingestados total: 0
Ingestados HUMAN: 0
Ingestados AUTO: 0
Modelo final: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt
Log de ingesta: c:\Users\Johnny\Desktop\IA\artifacts\retraining_ingest_logs\ingest_20260129_131548.json
